<a href="https://colab.research.google.com/github/KevinYih/BigDataDemo/blob/main/Copy_of_Spark's_Collaborative_Filtering_library_by_Kevin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Collaborative Filtering

source: https://colab.research.google.com/drive/1UWeDiyXiwDDqe7ksN2kt-myHsuSLObv8?usp=sharing

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=ae79f09049d963420f120860d99f649d3a42ffdd8135a7ff3ba1ecf55d64217c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 39.6 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1QtPy_HuIMSzhtYllT3-WeM3Sqg55wK_D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.training')

id='1ePqnsQTJRRvQcBoF2EhoPU8CU1i5byHK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.test')

id='1ncUBWdI5AIt3FDUJokbMqpHD2knd5ebp'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.item')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [ ]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("MovieLens.item", header=False, schema=schema_items)

In [ ]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [ ]:
items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [ ]:
# Compute the number of ratings in the training and test dataset.
training_count = training.count()
test_count = test.count()

print(f"Number of ratings in the training dataset: {training_count}")
print(f"Number of ratings in the test dataset: {test_count}")

# Compute the number of movies in the dataset
movie_count = items.select("item_id").distinct().count()

print(f"Number of movies in the dataset: {movie_count}")

Number of ratings in the training dataset: 80000
Number of ratings in the test dataset: 20000
Number of movies in the dataset: 1682


Using the training set, train a model with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

In [ ]:
# YOUR CODE HERE
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

# Create SparkSession
spark = SparkSession.builder.appName("ALSExample").getOrCreate()

# Load data
lines = spark.read.text("/content/MovieLens.training").rdd
# parts = lines.map(lambda row: row.value.split("::"))
# ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
#                                      rating=float(p[2]), timestamp=int(p[3])))

parts = lines.map(lambda row: row.value.split("\t"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))

ratings = spark.createDataFrame(ratingsRDD)

(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)


Now compute the RMSE on the test dataset.


In [ ]:
# YOUR CODE HERE
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.121900609777377


At this point, you can use the trained model to produce the top-K recommendations for each user. You have to produce the title of top movies, not just the ids

In [ ]:
# YOUR CODE HERE

# Produce the top-K recommendations for each user
topK = 10
userRecs = model.recommendForAllUsers(topK)

# Join with the items dataframe to get movie titles
userRecs = userRecs.withColumn("recommendations", explode("recommendations"))
userRecs = userRecs.select(col("userId"), col("recommendations.movieId"), col("recommendations.rating"))

# Join with items dataframe to get movie titles
userRecsWithTitles = userRecs.join(items, userRecs.movieId == items.item_id).select("userId", "movie", "rating")

# Show the top-K recommendations for each user
userRecsWithTitles.show(truncate=False)


+------+-----------------------------------------------------------+---------+
|userId|movie                                                      |rating   |
+------+-----------------------------------------------------------+---------+
|1     |Angel Baby (1995)                                          |7.623747 |
|1     |8 1/2 (1963)                                               |6.129052 |
|1     |Safe (1995)                                                |5.9240837|
|1     |Mina Tannenbaum (1994)                                     |5.91947  |
|1     |Orlando (1993)                                             |5.850461 |
|1     |Paradise Lost: The Child Murders at Robin Hood Hills (1996)|5.8177156|
|1     |Three Caballeros, The (1945)                               |5.7579355|
|1     |Shall We Dance? (1937)                                     |5.72886  |
|1     |Dead Man (1995)                                            |5.610992 |
|1     |Farewell My Concubine (1993)                

Add your own ratings to the dataset and use the model to produce top-K recommendations for yourself.

These are the steps that you need to take:


1.   Search the dataset for movies that you have watched before. You need the ids of those movies
2.   Add your ratings to the training dataset. You need to generate a unique user_id for yourself (a user_id that does not already exist in the dataset).
3.   Train the model using the new dataset.
4.   Produce top-K recommendation for youself.



In [ ]:
# YOUR CODE HERE


# 1. Search the dataset for movies that you have watched before. You need the ids of those movies
watched_movies = ["Toy Story (1995)", "GoldenEye (1995)", "Four Rooms (1995)"]
watched_movies_ids = items.filter(items.movie.isin(watched_movies)).select("item_id", "movie").collect()
print("Watched movies and their IDs:")
for row in watched_movies_ids:
    print(f"{row['movie']}: {row['item_id']}")

# 2. You need to generate a unique user_id for yourself (a user_id that does not already exist in the dataset)
my_user_id = 9999

# 2. Add your ratings to the training dataset.
my_ratings_data = [
    (my_user_id, row['item_id'], 5, int(np.datetime64('now').astype(int) // 10**9))  # 假设你给这些电影的评分都是 5
    for row in watched_movies_ids
]

my_ratings_schema = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)
])

my_ratings_df = spark.createDataFrame(my_ratings_data, my_ratings_schema)

# 2. Add your ratings to the training dataset.
new_training = training.union(my_ratings_df)

# 3. Train the model using the new dataset.
new_model = als.fit(new_training)

# 4. Generate top-K recommendations for yourself
topK = 10
my_recs = new_model.recommendForUserSubset(spark.createDataFrame([(my_user_id,)], ["userId"]), topK)

# Join with the items dataframe to get movie titles
my_recs = my_recs.withColumn("recommendations", explode("recommendations"))
my_recs = my_recs.select(col("userId"), col("recommendations.movieId"), col("recommendations.rating"))

# change the column name “movieId” to “item_id” for join
my_recs = my_recs.withColumnRenamed("movieId", "item_id")

# Join with items dataframe to get movie titles
my_recs_with_titles = my_recs.join(items, my_recs.item_id == items.item_id).select("userId", "movie", "rating")

# Show the top-K recommendations for yourself
my_recs_with_titles.show(truncate=False)


Watched movies and their IDs:
Toy Story (1995): 1
GoldenEye (1995): 2
Four Rooms (1995): 3
+------+--------------------------------------------------------------------+---------+
|userId|movie                                                               |rating   |
+------+--------------------------------------------------------------------+---------+
|9999  |Ruling Class, The (1972)                                            |6.533584 |
|9999  |Angel Baby (1995)                                                   |6.477675 |
|9999  |Bread and Chocolate (Pane e cioccolata) (1973)                      |6.2941732|
|9999  |Haunted World of Edward D. Wood Jr., The (1995)                     |6.2030606|
|9999  |Burnt By the Sun (1994)                                             |6.181601 |
|9999  |Pather Panchali (1955)                                              |6.102566 |
|9999  |Sum of Us, The (1994)                                               |6.0794644|
|9999  |Supercop (1992)      